Продолжим работу с данными, которые были использованы в ДЗ2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

### Задание 0: 
Выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

In [51]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
import catboost as catb
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm
import shap


%matplotlib inline

In [52]:
train = pd.read_csv("assignment_2_train.csv")
test = pd.read_csv("assignment_2_test.csv")

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))

train.shape = 180000 rows, 394 cols
test.shape = 100001 rows, 394 cols


In [53]:
train.sort_values("TransactionID", inplace=True)
X_train = train.drop("isFraud", axis=1)
y_train = train["isFraud"]

In [54]:
test.sort_values("TransactionID", inplace=True)
X_test = test.drop("isFraud", axis=1)
y_test = test["isFraud"]

In [55]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, shuffle=False, test_size=0.3, random_state=42)

In [56]:
numerical_features = X_train.select_dtypes(exclude=["object"])
numerical_features = numerical_features.columns.tolist()

In [57]:
def fit_eval(X_train, X_valid, X_test, y_train, y_valid, feats=numerical_features):
    eval_set = [(X_valid[feats], y_valid)]
    model = xgb.XGBClassifier(n_estimators=100, n_jobs=8)
    model.fit(X_train[feats],
            y_train,
            eval_metric='auc',
            eval_set=eval_set,
            early_stopping_rounds=5)
    
    y_train_pred = model.predict_proba(X_train[feats])
    print(f"Train score (ROC_AUC): {roc_auc_score(y_train, y_train_pred[:, 1])}")
    y_valid_pred = model.predict_proba(X_valid[feats])
    print(f"Valid score (ROC_AUC): {roc_auc_score(y_valid, y_valid_pred[:, 1])}")
    y_test_pred = model.predict_proba(X_test[feats])
    print(f"Test  score (ROC_AUC): {roc_auc_score(y_test, y_test_pred[:, 1])}")
    
    return model

In [58]:
base_model = fit_eval(X_train, X_valid, X_test, y_train, y_valid)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[0]	validation_0-auc:0.69945
[1]	validation_0-auc:0.79227
[2]	validation_0-auc:0.83576
[3]	validation_0-auc:0.83572
[4]	validation_0-auc:0.83670
[5]	validation_0-auc:0.83706
[6]	validation_0-auc:0.83939
[7]	validation_0-auc:0.84944
[8]	validation_0-auc:0.85100
[9]	validation_0-auc:0.85545
[10]	validation_0-auc:0.83308
[11]	validation_0-auc:0.81001
[12]	validation_0-auc:0.82093
[13]	validation_0-auc:0.83149
[14]	validation_0-auc:0.84480
Train score (ROC_AUC): 0.8900934092842951
Valid score (ROC_AUC): 0.8554492591505862
Test  score (ROC_AUC): 0.8401148107316188


### Задание 1: 
Признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [59]:
def process_transactionDT(X):
    base_date = pd.to_datetime('2017-12-01')
    X["TransactionDT"] = base_date + pd.to_timedelta(X["TransactionDT"], unit='s')
    X["TransactionDT_Year"] = X["TransactionDT"].dt.year
    X["TransactionDT_Month"] = X["TransactionDT"].dt.month
    X["TransactionDT_WeekDay"] = X["TransactionDT"].dt.weekday
    X["TransactionDT_Day"] = X["TransactionDT"].dt.day
    X["TransactionDT_Hour"] = X["TransactionDT"].dt.hour
    X["TransactionDT"] = (X["TransactionDT"].astype('int64')/1000000000).astype('int64')
    return X

In [60]:
X_train = process_transactionDT(X_train)
X_valid = process_transactionDT(X_valid)
X_test = process_transactionDT(X_test)

In [61]:
all_features = numerical_features + ["TransactionDT_Year", "TransactionDT_Month", "TransactionDT_WeekDay", "TransactionDT_Day", "TransactionDT_Hour"]

In [62]:
model = fit_eval(X_train, X_valid, X_test, y_train, y_valid, feats=all_features)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[0]	validation_0-auc:0.69945
[1]	validation_0-auc:0.79227
[2]	validation_0-auc:0.83578
[3]	validation_0-auc:0.83574
[4]	validation_0-auc:0.83675
[5]	validation_0-auc:0.83714
[6]	validation_0-auc:0.83946
[7]	validation_0-auc:0.84949
[8]	validation_0-auc:0.85104
[9]	validation_0-auc:0.85555
[10]	validation_0-auc:0.83228
[11]	validation_0-auc:0.83645
[12]	validation_0-auc:0.84455
[13]	validation_0-auc:0.85653
[14]	validation_0-auc:0.83950
[15]	validation_0-auc:0.84881
[16]	validation_0-auc:0.85451
[17]	validation_0-auc:0.85962
[18]	validation_0-auc:0.86272
[19]	validation_0-auc:0.86613
[20]	validation_0-auc:0.86897
[21]	validation_0-auc:0.87116
[22]	validation_0-auc:0.87108
[23]	validation_0-auc:0.87446
[24]	validation_0-auc:0.87464
[25]	validation_0-auc:0.87523
[26]	validation_0-auc:0.87597
[27]	validation_0-auc:0.87641
[28]	validation_0-auc:0.87684
[29]	validation_0-auc:0.87662
[30]	validation_0-auc:0.87683
[31]	validation_0-auc:0.87722
[32]	validation_0-auc:0.87433
[33]	validation_0-au

Результат совсем немного, но стал лучше. Подозреваю, что это связано с тем, что хоть алгоритм может вывести какие-то временные зависимости (на выходных или перед праздниками, к примеру, поведение и обычных клиентов, и мошенников может меняться), но алгоритму это легче сделать с выделенными признаками.

### Задание 2: 
Сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

### Задание 3: 
Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [63]:
def process_cards_feats_enc(data):
    new_feats = []
    for feat in ["card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2"]:
        freq_encoder = data[feat].value_counts(normalize=True)
        data[f"{feat}_freq_enc"] = data[feat].map(freq_encoder)
        new_feats += [f"{feat}_freq_enc"]
    return data, new_feats

In [64]:
X_train, new_feats = process_cards_feats_enc(X_train)
X_valid, new_feats = process_cards_feats_enc(X_valid)
X_test, new_feats = process_cards_feats_enc(X_test)

In [65]:
new_feats

['card1_freq_enc',
 'card2_freq_enc',
 'card3_freq_enc',
 'card4_freq_enc',
 'card5_freq_enc',
 'card6_freq_enc',
 'addr1_freq_enc',
 'addr2_freq_enc']

In [66]:
all_features += new_feats

In [67]:
model = fit_eval(X_train, X_valid, X_test, y_train, y_valid, feats=all_features)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[0]	validation_0-auc:0.69880
[1]	validation_0-auc:0.79234
[2]	validation_0-auc:0.83462
[3]	validation_0-auc:0.83274
[4]	validation_0-auc:0.83408
[5]	validation_0-auc:0.83608
[6]	validation_0-auc:0.83745
[7]	validation_0-auc:0.84590
[8]	validation_0-auc:0.85197
[9]	validation_0-auc:0.85872
[10]	validation_0-auc:0.83012
[11]	validation_0-auc:0.84714
[12]	validation_0-auc:0.81837
[13]	validation_0-auc:0.82810
Train score (ROC_AUC): 0.8943290986990793
Valid score (ROC_AUC): 0.8587190485277483
Test  score (ROC_AUC): 0.8474646251738036


Немного увеличилось качество модели, похоже, что введённые признаки полезны.

### Задание 4: 
Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

### Задание 5: 
Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

### Задание 6: 
выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt